In [5]:
import re

class Tokenizer:
    def __init__(self):
        self.tokens = []

    def tokenize_input(self, input_string):
        # Define regex patterns for tokens
        patterns = [
            (r'def', 'DEF'),                # Match '='
            (r'if', 'IF'),                   # Match 'if'
            (r'else', 'ELSE'),               # Match 'else'
            (r'while', 'WHILE'),             # Match 'while'
            (r'return', 'RETURN'),           # Match 'return'
            (r'print', 'PRINT'),                   # Match 'def'
            (r'[a-zA-Z_][a-zA-Z0-9_]*', 'IDENTIFIER'),  # Match identifiers
            (r'\d+', 'NUMBER'),              # Match numbers
            (r'\(', 'LPAREN'),               # Match '('
            (r'\)', 'RPAREN'),               # Match ')'
            (r':', 'COLON'),                 # Match ':'
            (r',', 'COMMA'),                 # Match ','
            (r'\+', 'PLUS'),                 # Match '+'
            (r'-', 'MINUS'),                 # Match '-'
            (r'\*', 'MULTIPLY'),             # Match '*'
            (r'/', 'DIVIDE'),                # Match '/'
            (r'=', 'ASSIGN'),           # Match 'print'
            (r'>', 'GREATER_THAN'),          # Match '>'
            (r'<', 'LESS_THAN'),             # Match '<'
            (r'\n', 'NEWLINE'),              # Match newline
            (r'\s+', None),                  # Match whitespace (ignore)
            (r'.', 'INVALID')                # Match any other character (invalid)
        ]

        # Tokenize input string
        while input_string:
            found_match = False
            for pattern, token_type in patterns:
                match = re.match(pattern, input_string)
                if match:
                    found_match = True
                    if token_type:
                        self.tokens.append((match.group(), token_type))
                    input_string = input_string[match.end():]
                    break
            if not found_match:
                raise SyntaxError("Invalid input")

    def accept_input(self):
        try:
            input_string = input("Enter your code:\n")
            self.tokenize_input(input_string)
            print("Tokens:", self.tokens)
        except SyntaxError as e:
            print("Error:", e)

# Example usage
tokenizer = Tokenizer()
tokenizer.accept_input()

Enter your code:
def add(a, b):     return a + b  def subtract(a, b):     return a - b  def multiply(a, b):     return a * b  def divide(a, b):     return a / b  x = 10 y = 5 if x > y:     print("x is greater than y") else:     print("x is not greater than y")  while x > 0:     print(x)     x = x - 1  print("End of program")
Tokens: [('def', 'DEF'), ('add', 'IDENTIFIER'), ('(', 'LPAREN'), ('a', 'IDENTIFIER'), (',', 'COMMA'), ('b', 'IDENTIFIER'), (')', 'RPAREN'), (':', 'COLON'), ('return', 'RETURN'), ('a', 'IDENTIFIER'), ('+', 'PLUS'), ('b', 'IDENTIFIER'), ('def', 'DEF'), ('subtract', 'IDENTIFIER'), ('(', 'LPAREN'), ('a', 'IDENTIFIER'), (',', 'COMMA'), ('b', 'IDENTIFIER'), (')', 'RPAREN'), (':', 'COLON'), ('return', 'RETURN'), ('a', 'IDENTIFIER'), ('-', 'MINUS'), ('b', 'IDENTIFIER'), ('def', 'DEF'), ('multiply', 'IDENTIFIER'), ('(', 'LPAREN'), ('a', 'IDENTIFIER'), (',', 'COMMA'), ('b', 'IDENTIFIER'), (')', 'RPAREN'), (':', 'COLON'), ('return', 'RETURN'), ('a', 'IDENTIFIER'), ('*', 'MULT